In [30]:
%matplotlib inline
%load_ext memory_profiler
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.autograd import Variable
import bcolz
from tqdm import tqdm,tnrange, tqdm_notebook
tqdm.monitor_interval = 0
from itertools import islice
import matplotlib.pyplot as plt
import numpy as np
#import main; reload(main)
from memory_profiler import profile
from ipywidgets import widgets
from IPython.display import display

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [43]:
batch=120
workers=8
trainset = torchvision.datasets.ImageFolder('train',
                                           transform=transforms.Compose([
                                               transforms.RandomResizedCrop(224),
                                               #transforms.RandomHorizontalFlip(),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                                           ]))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch,
                                          shuffle=True, num_workers=workers,pin_memory=True)
validset = torchvision.datasets.ImageFolder('valid',
                                           transform=transforms.Compose([
                                               transforms.RandomResizedCrop(224),
                                               #transforms.RandomHorizontalFlip(),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                                           ]))
validloader = torch.utils.data.DataLoader(validset, batch_size=batch,
                                          shuffle=True, num_workers=workers)
print(trainset.classes,trainset.imgs[0],len(trainset.imgs))

['cat', 'dog'] ('train/cat/cat.0.jpg', 0) 23000


In [44]:
if 'net' in locals():
    del net
net=models.vgg16(pretrained=True)
#net=torchvision.models.resnet18()
for param in net.parameters():
    param.requires_grad = False
# Replace the last fully-connected layer
# Parameters of newly constructed modules have requires_grad=True by default
net.fc = nn.Linear(512, 2)

In [45]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss().cuda()
# Optimize only the classifier
optimizer = optim.Adam(net.fc.parameters(), lr=1e-3)

In [51]:
import threading
from IPython.core.debugger import set_trace

def train(network, loader, vloader, breaking):
    with tnrange(2, desc="Training", unit="epoch") as epochbar:
        for epoch in epochbar:  # loop over the dataset multiple times
            with tqdm_notebook(
                      #total=len(loader.dataset.imgs)/loader.batch_size,
                      desc="Epoch %d" % (epoch + 1),
                      unit="batch",
                      leave=True) as batchbar:
                training_loss=0.0
                for i, data in enumerate(loader, 0):
                    #print("%d, data length: %d " % (i,len(data[0])))
                    if breaking.value:
                        print('Aborted Training')
                        return

                    # get the inputs
                    inputs, labels = data
                    inputs, labels = Variable(inputs, requires_grad=True).cuda(), Variable(labels).cuda(async=True)

                    # zero the parameter gradients
                    optimizer.zero_grad()
                    
                    # forward + backward + optimize
                    outputs = network(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    
                    # update text
                    training_loss = loss.cpu()[0]
                    batchbar.set_postfix(loss=training_loss/loader.batch_size,refresh=False)
                    batchbar.update()
                
                validation_loss=0.0
                for i, data in enumerate(vloader,0):
                    batchbar.set_postfix(loss=training_loss,validation_loss="validating... %d"%i)
                     # get the inputs
                    inputs, labels = data
                    inputs, labels = Variable(inputs, requires_grad=False).cuda(), Variable(labels).cuda(async=True)

                    # forward
                    outputs = network(inputs)
                    loss = criterion(outputs, labels)
                    validation_loss += loss.cpu()[0]
                
                # update text
                batchbar.set_postfix(loss=training_loss,validation_loss=validation_loss/len(vloader.dataset.imgs))

    print('Finished Training')

def runtest():
    breaking=widgets.ToggleButton(description="Break")
    display(breaking)
    bg=threading.Thread(target=train,args=(net.cuda(),trainloader,validloader,breaking,))
    bg.start()

cudnn.benchmark=True
runtest()
#train(net.cuda(),trainloader,validloader,widgets.ToggleButton(description="Break"))

ToggleButton(value=False, description='Break')

HBox(children=(IntProgress(value=0, description='Training', max=2), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', description='Epoch 1', max=1), HTML(value='')))